In [26]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import multiprocess

In [3]:
monsters = pd.read_csv('../transform/monster_lb.csv')
monsters.head()

,Nome,Tipo,Tamanho,Tendência,PV_médio,PV_dados,CA,Deslocamento,Força,mod_força,...,mod_inteligência,Sabedoria,mod_sabedoria,Carisma,mod_carisma,Infos,Nível_desafio,XP,Ações,Dano_médio
0,AARAKOCRA,Humanoide,Médio,neutro e bom,13,3d8,12,6.0,10.0,0.0,...,0.0,12.0,1.0,11.0,0.0,Perícias Percepção +5 Sentidos Percepção passi...,1/4,50.0,Garra. Ataque Corpo-a-Corpo com Arma: +4 para...,5.0
1,ABOCANHADOR MATRAQUEANTE,Aberração,Média,neutro,67,9d8 + 27,9,3.0,10.0,0.0,...,-4.0,10.0,0.0,6.0,-2.0,Imunidade a Condição caído Sentidos visão no ...,2,450.0,Ataques Múltiplos. O abocanhador matraqueante...,17.0
2,ABOLETE,Aberração,Grande,leal e mau,135,18d10 + 36,17,3.0,21.0,5.0,...,4.0,15.0,2.0,18.0,4.0,"Testes de Resistência Con +6, Int +8, Sab +6 ...",10,5900.0,"O abolete pode realizar 3 ações lendárias, e...",15.0
3,DEVA,Celestial,Médio,leal e bom,136,16d8 + 64,17,9.0,18.0,4.0,...,3.0,20.0,5.0,20.0,5.0,Ataques Múltiplos. O deva realiza dois ataque...,10,5900.0,"Celestial Médio, leal e bom Classe de Armadu...",25.0
4,PLANETÁRIO,Celestial,Grande,leal e bom,200,16d10 + 112,19,12.0,24.0,7.0,...,4.0,22.0,6.0,25.0,7.0,"Testes de Resistência Con +12, Sab +11, Car +...",16,15000.0,Ataques Múltiplos. O planetário realiza dois a...,43.0


In [4]:
# base do dataframe
battles = pd.DataFrame({'n_players':'0', 'max_lvl_player':0, 'min_lvl_player':0, 
                       'max_for_player':0, 'min_for_player':0, 'max_int_player':0, 'min_int_player':0,
                        'max_des_player':0, 'min_des_player':0, 'max_car_player':0, 'min_car_player':0,
                        'max_sab_player':0, 'min_sab_player':0, 'max_con_player':0, 'min_con_player':0,
                        'max_pv_player':0, 'min_pv_player':0, 'max_ca_player':0, 'min_ca_player':0,
                       'max_dano_player':0, 'min_dano_player':0,
                       'n_creature':0, 'max_xp_creature':0, 'min_xp_creature':0, 
                       'max_for_creature':0, 'min_for_creature':0, 'max_int_creature':0, 'min_int_creature':0,
                        'max_des_creature':0, 'min_des_creature':0, 'max_car_creature':0, 'min_car_creature':0,
                        'max_sab_creature':0, 'min_sab_creature':0, 'max_con_creature':0, 'min_con_creature':0,
                        'max_pv_creature':0, 'min_pv_creature':0, 'max_ca_creature':0, 'min_ca_creature':0,
                       'max_dano_creature':0, 'min_dano_creature':'0', 'resultado_battle':1}, index = [0])
battles.drop(index = [0], inplace = True)
battles

,n_players,max_lvl_player,min_lvl_player,max_for_player,min_for_player,max_int_player,min_int_player,max_des_player,min_des_player,max_car_player,...,min_sab_creature,max_con_creature,min_con_creature,max_pv_creature,min_pv_creature,max_ca_creature,min_ca_creature,max_dano_creature,min_dano_creature,resultado_battle


In [13]:
# simulação que cria os personagens, define os monstros e simula a  batalha

def simulation(x):    
    
    import pandas as pd
    import numpy as np
    monsters = pd.read_csv('../transform/monster_lb.csv')
    
    def battle(players, creatures):
        vida_players = []
        ca_players = []
        dano_players = []
        mod_players = []
        for n in range(len(players)):
            vida_players.append(players[n][-4])
            ca_players.append(players[n][-3])
            dano_players.append(players[n][-2])
            mod_players.append(players[n][-1])

        vida_creatures = []
        ca_creatures = []
        dano_creatures = []
        mod_creatures = []
        for n in range(len(creatures)):
            vida_creatures.append(creatures[n][-4])
            ca_creatures.append(creatures[n][-3])
            dano_creatures.append(creatures[n][-2])
            mod_creatures.append(creatures[n][-1])

        first_attack = np.random.randint(0,2,1)[0]

        tot_vida_players = sum(vida_players)
        tot_vida_creatures = sum(vida_creatures)

        while tot_vida_players > 0 and tot_vida_creatures > 0:
    #         print('vida antes players', tot_vida_players)
    #         print('vida antes creatures', tot_vida_creatures)
            if first_attack == 1:

                for i in range(len(vida_players)):
                    d20 = np.random.randint(1,21,1)[0] + min(mod_players)
                    if d20 >= max(ca_creatures):
                        tot_vida_creatures -= dano_players[i]

                for i in range(len(vida_creatures)):
                    d20 = np.random.randint(1,21,1)[0] + min(mod_creatures)
                    if d20 >= max(ca_players):
                        tot_vida_players -= dano_creatures[i]

            else:

                for i in range(len(vida_creatures)):
                    d20 = np.random.randint(1,21,1)[0] + min(mod_creatures)
                    if d20 >= max(ca_players):
                        tot_vida_players -= dano_creatures[i]

                for i in range(len(vida_players)):
                    d20 = np.random.randint(1,21,1)[0] + min(mod_players)
                    if d20 >= max(ca_creatures):
                        tot_vida_creatures -= dano_players[i]

    #         print('vida depois players', tot_vida_players)
    #         print('vida depois creatures', tot_vida_creatures)

        if tot_vida_players > 0:
            resultado = 1
    #         print('Vitória players\n')

        else:
            resultado = 0
    #         print('Vitória monstros\n')
        return resultado


    
    n_players = np.random.randint(1, 5, 1)[0]
    
    lvl = np.random.randint(2,20,1)[0]
    
    players = []
    for j in range(n_players):
        players.append([lvl+np.random.randint(-1,2,1)[0]])
        
        classe = np.random.randint(1,6,1)[0]
        players[j].append(classe)
        
        dados = [sum(np.random.randint(1,7,3)), sum(np.random.randint(1,7,3)), sum(np.random.randint(1,7,3)), sum(np.random.randint(1,7,3)), sum(np.random.randint(1,7,3)), sum(np.random.randint(1,7,3))]
        dados.sort()
        
        if classe == 1:
            forca = dados[-1] + players[j][0]-1
            players[j].append(forca)
            
            inte = dados[-4]
            players[j].append(inte)
            
            des = dados[-3]
            players[j].append(des)
            
            car = dados[-6]
            players[j].append(car)
            
            sab = dados[-5]
            players[j].append(sab)
            
            con = dados[-2]
            players[j].append(con)
            
            modcon = con -10
            if modcon >= 0:
                modcon = int(modcon/2)
            else:
                modcon = int((modcon-0.5)/2)
            
            pv = 12 + (7+modcon)*(players[j][0]-1)
            players[j].append(pv)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            if moddes > 2:
                moddes = 2
            
            ca = 14 + moddes
            players[j].append(ca)
            
            modfor = forca - 10
            if modfor >= 0:
                modfor = int(modfor/2)
            else:
                modfor = int((modfor-0.5)/2)
            
            dano = 7 + modfor*3
            players[j].append(dano)
            players[j].append(modfor)
            
            
            
            
        elif classe == 2:
            forca = dados[-6]
            players[j].append(forca)
            
            inte = dados[-3]
            players[j].append(inte)
            
            des = dados[-5]
            players[j].append(des)
            
            car = dados[-4]
            players[j].append(car)
            
            sab = dados[-1] + players[j][0]-1
            players[j].append(sab)
            
            con = dados[-2]
            players[j].append(con)
            
            modcon = con -10
            if modcon >= 0:
                modcon = int(modcon/2)
            else:
                modcon = int((modcon-0.5)/2)
            
            pv = 7 + (5+modcon)*(players[j][0]-1)
            players[j].append(pv)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            if moddes > 2:
                moddes = 2
            
            ca = 12 + moddes
            players[j].append(ca)
            
            modsab = sab - 10
            if modsab >= 0:
                modsab = int(modsab/2)
            else:
                modsab = int((modsab-0.5)/2)
            
            dano = 7 + modsab*4
            players[j].append(dano)
            players[j].append(modsab)
            
            
            
            
        elif classe == 3:
            forca = dados[-6]
            players[j].append(forca)
            
            inte = dados[-3]
            players[j].append(inte)
            
            des = dados[-5]
            players[j].append(des)
            
            car = dados[-1] + players[j][0]-1
            players[j].append(car)
            
            sab = dados[-4]
            players[j].append(sab)
            
            con = dados[-2]
            players[j].append(con)
            
            modcon = con -10
            if modcon >= 0:
                modcon = int(modcon/2)
            else:
                modcon = int((modcon-0.5)/2)
            
            pv = 6 + (4+modcon)*(players[j][0]-1)
            players[j].append(pv)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            if moddes > 2:
                moddes = 2
            
            ca = 12 + moddes
            players[j].append(ca)
            
            modcar = car - 10
            if modcar >= 0:
                modcar = int(modcar/2)
            else:
                modcar = int((modcar-0.5)/2)
            
            dano = 4 + modcar*6
            players[j].append(dano)
            players[j].append(modcar)
                
        
        elif classe == 4:
            forca = dados[-1] + players[j][0]-1
            players[j].append(forca)
            
            inte = dados[-4]
            players[j].append(inte)
            
            des = dados[-3]
            players[j].append(des)
            
            car = dados[-6]
            players[j].append(car)
            
            sab = dados[-5]
            players[j].append(sab)
            
            con = dados[-2]
            players[j].append(con)
            
            modcon = con -10
            if modcon >= 0:
                modcon = int(modcon/2)
            else:
                modcon = int((modcon-0.5)/2)
            
            pv = 10 + (6+modcon)*(players[j][0]-1)
            players[j].append(pv)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            if moddes > 2:
                moddes = 2
            
            ca = 15 + moddes
            players[j].append(ca)
            
            modfor = forca - 10
            if modfor >= 0:
                modfor = int(modfor/2)
            else:
                modfor = int((modfor-0.5)/2)
            
            dano = 7 + modfor*3
            players[j].append(dano)
            players[j].append(modfor)
            
            
        elif classe == 5:
            forca = dados[-3]
            players[j].append(forca)
            
            inte = dados[-5]
            players[j].append(inte)
            
            des = dados[-1] + players[j][0]-1
            players[j].append(des)
            
            car = dados[-6]
            players[j].append(car)
            
            sab = dados[-2]
            players[j].append(sab)
            
            con = dados[-4]
            players[j].append(con)
            
            modcon = con -10
            if modcon >= 0:
                modcon = int(modcon/2)
            else:
                modcon = int((modcon-0.5)/2)
            
            pv = 10 + (6+modcon)*(players[j][0]-1)
            players[j].append(pv)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            if moddes > 2:
                moddes = 2
            
            ca = 12 + moddes
            players[j].append(ca)
            
            moddes = des - 10
            if moddes >= 0:
                moddes = int(moddes/2)
            else:
                moddes = int((moddes-0.5)/2)
            
            dano = 7 + moddes*3
            players[j].append(dano)
            players[j].append(moddes)
    
    
    
    n_monst = np.random.randint(1,5,1)[0]
    
    creatures = []
    intermediario = monsters.reset_index()
    niv = list(intermediario['XP'].unique())
    niv.sort()
    
    nivel_creat = np.random.randint(1, (len(niv)-1),1)[0]
    
    new_monst = intermediario[intermediario.XP.isin([niv[nivel_creat-1], niv[nivel_creat], niv[nivel_creat+1]])]
    id_monst = list(new_monst['index'].unique())
    for k in range(n_monst):
        id_c = id_monst[np.random.randint(0, len(id_monst), 1)[0]]
        creatures.append([])
        
        XP = new_monst.loc[id_c, 'XP']
        creatures[k].append(XP)
        
        Força = new_monst.loc[id_c, 'Força']
        creatures[k].append(Força)
        
        Inteligência = new_monst.loc[id_c, 'Inteligência']
        creatures[k].append(Inteligência)
        
        Destreza = new_monst.loc[id_c, 'Destreza']
        creatures[k].append(Destreza)
        
        Carisma = new_monst.loc[id_c, 'Carisma']
        creatures[k].append(Carisma)
        
        Sabedoria = new_monst.loc[id_c, 'Sabedoria']
        creatures[k].append(Sabedoria)
        
        Constituição = new_monst.loc[id_c, 'Constituição']
        creatures[k].append(Constituição)
        
        PV_médio = new_monst.loc[id_c, 'PV_médio']
        creatures[k].append(PV_médio)
        
        CA = new_monst.loc[id_c, 'CA']
        creatures[k].append(CA)
        
        Dano_médio = new_monst.loc[id_c, 'Dano_médio']
        creatures[k].append(Dano_médio)
        
        mod_list = [new_monst.loc[id_c, 'mod_força'], new_monst.loc[id_c, 'mod_inteligência'], new_monst.loc[id_c, 'mod_destreza'], new_monst.loc[id_c, 'mod_carisma'], new_monst.loc[id_c, 'mod_sabedoria'], new_monst.loc[id_c, 'mod_constituição'], ]
        creatures[k].append(max(mod_list))
        
    resultado = battle(players, creatures)
    
    lvl_players = []
    for_players = []
    int_players = []
    car_players = []
    des_players = []
    sab_players = []
    con_players = []
    vida_players = []
    ca_players = []
    dano_players = []
    for n in range(len(players)):
        lvl_players.append(players[n][-12])
        for_players.append(players[n][-10])
        int_players.append(players[n][-9])
        des_players.append(players[n][-8])
        car_players.append(players[n][-7])
        sab_players.append(players[n][-6])
        con_players.append(players[n][-5])
        vida_players.append(players[n][-4])
        ca_players.append(players[n][-3])
        dano_players.append(players[n][-2])

        
    xp_creatures = []
    for_creatures = []
    int_creatures = []
    car_creatures = []
    des_creatures = []
    sab_creatures = []
    con_creatures = []
    vida_creatures = []
    ca_creatures = []
    dano_creatures = []
    for n in range(len(creatures)):
        xp_creatures.append(creatures[n][-11])
        for_creatures.append(creatures[n][-10])
        int_creatures.append(creatures[n][-9])
        des_creatures.append(creatures[n][-8])
        car_creatures.append(creatures[n][-7])
        sab_creatures.append(creatures[n][-6])
        con_creatures.append(creatures[n][-5])
        vida_creatures.append(creatures[n][-4])
        ca_creatures.append(creatures[n][-3])
        dano_creatures.append(creatures[n][-2])
        
    battles = pd.DataFrame([{'n_players':len(players), 'max_lvl_player': max(lvl_players), 'min_lvl_player': min(lvl_players), 
                       'max_for_player':max(for_players), 'min_for_player': min(for_players), 'max_int_player':max(int_players), 'min_int_player': min(int_players),
                        'max_des_player': max(des_players), 'min_des_player': min(des_players), 'max_car_player': max(car_players), 'min_car_player': min(car_players),
                        'max_sab_player': max(sab_players), 'min_sab_player': min(sab_players), 'max_con_player': max(con_players), 'min_con_player': min(con_players),
                        'max_pv_player': max(vida_players), 'min_pv_player':min(vida_players), 'max_ca_player':max(ca_players), 'min_ca_player':min(ca_players),
                       'max_dano_player':max(dano_players), 'min_dano_player':min(dano_players),
                       'n_creature':len(creatures), 'max_xp_creature':max(xp_creatures), 'min_xp_creature': min(xp_creatures), 
                       'max_for_creature':max(for_creatures), 'min_for_creature': min(for_creatures), 'max_int_creature':max(int_creatures), 'min_int_creature': min(int_creatures),
                        'max_des_creature':max(des_creatures), 'min_des_creature': min(des_creatures), 'max_car_creature':max(car_creatures), 'min_car_creature': min(car_creatures),
                        'max_sab_creature':max(sab_creatures), 'min_sab_creature': min(sab_creatures), 'max_con_creature':max(con_creatures), 'min_con_creature': min(con_creatures),
                        'max_pv_creature':max(vida_creatures), 'min_pv_creature': min(vida_creatures), 'max_ca_creature':max(ca_creatures), 'min_ca_creature': min(ca_creatures),
                       'max_dano_creature':max(dano_creatures), 'min_dano_creature': min(dano_creatures), 'resultado_battle': resultado}])
    return battles

In [14]:
pool = multiprocess.Pool(8)

In [18]:
results = pool.map(simulation, range(300000))

In [25]:
pool.terminate()

In [32]:
df = pd.concat(results)

In [34]:
df

,n_players,max_lvl_player,min_lvl_player,max_for_player,min_for_player,max_int_player,min_int_player,max_des_player,min_des_player,max_car_player,...,min_sab_creature,max_con_creature,min_con_creature,max_pv_creature,min_pv_creature,max_ca_creature,min_ca_creature,max_dano_creature,min_dano_creature,resultado_battle
0,1,20,20,7,7,12,12,7,7,10,...,6.0,16.0,15.0,85,45,12,7,24.0,11.0,0
0,4,17,16,33,4,12,9,13,7,30,...,25.0,26.0,26.0,243,243,21,21,49.0,49.0,1
0,2,2,2,11,7,11,9,13,8,9,...,16.0,22.0,22.0,161,161,17,17,15.0,15.0,0
0,1,2,2,9,9,8,8,13,13,8,...,15.0,26.0,25.0,367,243,22,21,49.0,28.0,0
0,3,13,11,27,12,15,11,14,11,14,...,13.0,26.0,26.0,333,333,20,20,26.0,26.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4,12,10,26,8,13,9,13,8,25,...,13.0,21.0,20.0,195,189,19,18,21.0,15.0,1
0,3,17,16,31,4,14,11,13,4,13,...,13.0,21.0,14.0,195,172,19,18,21.0,14.0,0
0,4,11,10,27,5,13,9,11,5,24,...,10.0,12.0,10.0,11,7,12,10,8.0,5.0,1
0,4,14,12,26,7,12,9,11,7,24,...,13.0,26.0,24.0,333,300,20,19,43.0,26.0,0


In [35]:
df['resultado_battle'].mean()

0.4408

In [38]:
df.to_csv('battle_simulation.csv', index = False)